In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load the data
spacex_csv_file = r"C:\Users\reab5\Downloads\spacex_launch_geo.csv"
spacex_df = pd.read_csv(spacex_csv_file)

# Check column names to ensure they match
print(spacex_df.columns)

# Initialize the Dash app
app = dash.Dash(__name__)

# Task 1: Add a Launch Site Drop-down Input Component
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Dashboard'),

    # Dropdown for Launch Sites
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                 ],
                 value='ALL',
                 placeholder='Select a Launch Site',
                 searchable=True
                 ),

    # Pie chart for success counts
    dcc.Graph(id='success-pie-chart'),

    html.Br(),

    # Range slider for payload
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={i: f'{i}' for i in range(0, 10001, 2500)},
                    value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]),

    # Scatter plot for payload vs. success
    dcc.Graph(id='success-payload-scatter-chart'),
])


# Task 2: Add a Callback Function to Render Success Pie Chart Based on Selected Site Dropdown
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        filtered_df = spacex_df
        fig = px.pie(filtered_df, values='class',
                     names='Launch Site',
                     title='Total Success Launches by Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, names='class',
                     title=f'Total Success Launches for site {selected_site}')
    return fig


# Task 3 & 4: Add Callback Function for Payload Slider and Update Scatter Plot
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_plot(selected_site, payload_range):
    low, high = payload_range
    print(f"Payload range: {low} - {high}")  # Debug statement
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]

    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    print(filtered_df.head())  # Debug statement to check filtered data

    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='Booster Version',
                     title='Correlation between Payload and Success for All Sites')
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8050, host='127.0.0.1')